<a href="https://colab.research.google.com/github/Factians/Project/blob/main/Q%26A_Chatbot_with_Vector_Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install required packages
!pip install fastapi uvicorn faiss-cpu langchain langchain-huggingface langchain-groq sentence-transformers requests nest_asyncio pyngrok

#Import Libraries & Setup
import os
import zipfile
import nest_asyncio
import uvicorn
import faiss
import numpy as np
import requests

from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

# Allow nested asyncio loop (needed in Colab)
nest_asyncio.apply()

# ─── Setting the Tokens ───────────────────────────────────────────────────────
NGROK_AUTH_TOKEN = "2unJNaCtCotO73thkuK2nQBIcDK_7F6bxERpE6aQnA3eRv91N"
GROQ_API_KEY = "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"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# ─── Create FastAPI App ─────────────────────────────────────────────────────
app = FastAPI(title="Chatbot API", description="Chatbot with FAISS + Groq", version="1.0")

class ChatQuery(BaseModel):
    query: str

@app.post("/chat", summary="Chatbot Endpoint", description="Send a query to the chatbot")
def chat_endpoint(chat_query: ChatQuery):
    query = chat_query.query
    response = qa_chain.run(query)
    return {"query": query, "response": response}

@app.get("/")
def read_root():
    return {"message": "Welcome to the Chatbot API. Use /chat endpoint to interact."}

# ─── STEP 1: Extract ZIP File ───────────────────────────────────────────────
zip_path = "/content/demo_bot_data.zip"
extract_folder = "/content/demo_bot_data"

with zipfile.ZipFile(zip_path, mode='r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(" Extraction Done! Files available in:", extract_folder)

# ─── STEP 2: Parse & Chunk Markdown Files ───────────────────────────────────
data_dir = "/content/demo_bot_data/demo_bot_data/demo_bot_data/ubuntu-docs"

md_files = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".md"):
            md_files.append(os.path.join(root, file))
print(f"Total Markdown Files Found: {len(md_files)}")

# Chunk the text: 500-character chunks with 50-character overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = []
for file in md_files:
    with open(file, "r", encoding="utf-8") as f:
        content = f.read()
        chunks = text_splitter.split_text(content)
        documents.extend(chunks)
print(f"Total Chunks Created: {len(documents)}")

# ─── STEP 3: Create & Save FAISS Vector Store ───────────────────────────────
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"  # Model supporting PyTorch weights
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# Build vector store from texts
vector_store = FAISS.from_texts(documents, embeddings)
vector_store.save_local("ubuntu_docs_faiss")
print("Vector store saved successfully in folder 'ubuntu_docs_faiss'!")

# ─── STEP 4: Load FAISS Vector Store & Setup Chatbot ───────────────────────
vector_store = FAISS.load_local(
    "ubuntu_docs_faiss",
    embeddings,
    allow_dangerous_deserialization=True
)
print("Vector store loaded successfully!")

# Create Groq LLM instance (using ChatGroq from langchain_groq)
llm = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile"
)

# Build the RetrievalQA chain using the vector store retriever
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vector_store.as_retriever())
print("RetrievalQA chain created!")

#STEP 5: Expose FastAPI Server via Ngrok
public_url = ngrok.connect(addr="8000")
print("🔗 Public URL:", public_url.public_url)

# Run the FastAPI server using Uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)


✅ Extraction Done! Files available in: /content/demo_bot_data
📄 Total Markdown Files Found: 54
📜 Total Chunks Created: 656


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

INFO:     Started server process [730]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔗 Public URL: https://a2bd-34-169-232-119.ngrok-free.app
INFO:     163.116.195.120:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     163.116.195.120:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     163.116.195.120:0 - "GET / HTTP/1.1" 200 OK
INFO:     163.116.195.120:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     163.116.195.120:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     163.116.195.120:0 - "GET /openapi.json HTTP/1.1" 200 OK


<ipython-input-3-8d6a106f058f>:41: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


INFO:     163.116.195.120:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     163.116.195.120:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     163.116.195.120:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     163.116.195.120:0 - "POST /chat HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [730]
